In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Using TensorFlow backend.


In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG19(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 512)

In [8]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [9]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 .....

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s


[CV] C=1000.0 ........................................................
[CV] C=1000.0 ........................................................
[CV] ............................ C=1.0, score=0.927573, total=   2.9s
[CV] ............................ C=1.0, score=0.912214, total=   2.9s
[CV] C=1000.0 ........................................................
[CV] ............................ C=0.1, score=0.922392, total=   3.0s
[CV] ............................ C=1.0, score=0.916031, total=   2.9s
[CV] ........................... C=0.01, score=0.956743, total=   3.1s
[CV] C=10000.0 .......................................................
[CV] .......................... C=100.0, score=0.931298, total=   2.8s
[CV] ........................... C=0.01, score=0.941476, total=   3.3s
[CV] ............................ C=0.1, score=0.930114, total=   3.1s
[CV] C=10000.0 .......................................................
[CV] ............................ C=1.0, score=0.938931, total=   3.2s
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    3.5s


[CV] C=100000.0 ......................................................
[CV] C=100000.0 ......................................................
[CV] C=100000.0 ......................................................
[CV] C=100000.0 ......................................................
[CV] ........................... C=10.0, score=0.917303, total=   3.1s
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.940204, total=   3.4s
[CV] ........................... C=10.0, score=0.798982, total=   3.4s
[CV] ........................... C=10.0, score=0.914758, total=   3.3s
[CV] .......................... C=100.0, score=0.922392, total=   3.3s
[CV] C=1000000.0 .....................................................
[CV] C=1000000.0 .....................................................
[CV] C=1000000.0 .....................................................
[CV] C=1000000.0 .....................................................
[CV] C

...

[CV] .................. C=10000000000.0, score=0.932570, total=   1.8s
[CV] .................... C=100000000.0, score=0.923664, total=   2.8s
[CV] .................. C=10000000000.0, score=0.932570, total=   1.9s
[CV] .................... C=100000000.0, score=0.932570, total=   3.0s
[CV] ................... C=1000000000.0, score=0.932570, total=   2.6s
[CV] ................... C=1000000000.0, score=0.923664, total=   2.3s
[CV] .................. C=10000000000.0, score=0.928753, total=   2.2s
[CV] ................... C=1000000000.0, score=0.928753, total=   2.5s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   10.0s remaining:    0.8s


[CV] .................. C=10000000000.0, score=0.917408, total=   2.5s
[CV] .................. C=10000000000.0, score=0.923664, total=   2.5s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   10.4s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [10]:
top_model.cv_results_

{'mean_fit_time': array([ 2.8306251 ,  3.00461392,  2.9839396 ,  3.28615441,  3.32809863,
         3.41422892,  3.29678631,  3.18043003,  3.36499734,  3.01619878,
         2.51745625,  2.32610641,  2.18813534]),
 'mean_score_time': array([ 0.07410393,  0.05322781,  0.05244479,  0.07722478,  0.04666018,
         0.06705761,  0.05619869,  0.0682344 ,  0.09737577,  0.04509621,
         0.01902442,  0.02341318,  0.00535145]),
 'mean_test_score': array([ 0.93996439,  0.9310608 ,  0.92520987,  0.8987535 ,  0.90409565,
         0.93131519,  0.92953447,  0.92953447,  0.92953447,  0.93207835,
         0.92699059,  0.92699059,  0.92699059]),
 'mean_train_score': array([ 0.98142971,  0.98702626,  0.97602463,  0.94785283,  0.94479775,
         0.9866446 ,  0.98225668,  0.98225668,  0.98225668,  0.98658101,
         0.98422772,  0.98422772,  0.98422772]),
 'param_C': masked_array(data = [0.01 0.10000000000000001 1.0 10.0 100.0 1000.0 10000.0 100000.0 1000000.0
  10000000.0 100000000.0 1000000000.0 

In [11]:
top_model.grid_scores_

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93996, std: 0.00936, params: {'C': 0.01},
 mean: 0.93106, std: 0.00524, params: {'C': 0.10000000000000001},
 mean: 0.92521, std: 0.00984, params: {'C': 1.0},
 mean: 0.89875, std: 0.05067, params: {'C': 10.0},
 mean: 0.90410, std: 0.03533, params: {'C': 100.0},
 mean: 0.93132, std: 0.00619, params: {'C': 1000.0},
 mean: 0.92953, std: 0.00804, params: {'C': 10000.0},
 mean: 0.92953, std: 0.00804, params: {'C': 100000.0},
 mean: 0.92953, std: 0.00804, params: {'C': 1000000.0},
 mean: 0.93208, std: 0.00520, params: {'C': 10000000.0},
 mean: 0.92699, std: 0.00580, params: {'C': 100000000.0},
 mean: 0.92699, std: 0.00580, params: {'C': 1000000000.0},
 mean: 0.92699, std: 0.00580, params: {'C': 10000000000.0}]

In [12]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9400
